In [1]:
optim = require 'optim'
fullset = torch.load('newdata.dat')

In [2]:
itorch.image(fullset.data[1])
print(fullset.label[1])

111	


In [3]:
shuffle = torch.randperm(fullset.size)
shuffleset = fullset
for i=1, fullset.size do
    shuffleset.data[i] = fullset.data[shuffle[i]]
    shuffleset.label[i] = fullset.label[shuffle[i]]
    end
fullset = shuffleset

In [4]:
fullset

{
  data : DoubleTensor - size: 40134x15x15
  size : 40121
  label : DoubleTensor - size: 40134
}


In [5]:
for i=1,fullset.size do
    if(fullset.label[i]<=57) then
        fullset.label[i]=fullset.label[i]-47
    else
        fullset.label[i]=fullset.label[i]-97+11;
    end
end

In [6]:
trainset = {
    size = 32000,
    data = fullset.data[{{1,32000}}]:double(),
    label = fullset.label[{{1,32000}}]
}

In [7]:
validationset = {
    size = 4121,
    data = fullset.data[{{32001,36121}}]:double(),
    label = fullset.label[{{32001,36121}}]
}

In [8]:
itorch.image(trainset.data[1])
print(trainset.label[1])

25	


In [9]:
testset = {
    size = 4000,
    data = fullset.data[{{36122,40121}}]:double(),
    label = fullset.label[{{36122,40121}}]
}

In [171]:
nn = require 'nn'
model = nn.Sequential()
model:add(nn.Reshape(225))

In [172]:
model:add(nn.Linear(225,700))
model:add(nn.Tanh())
--model:add(nn.Linear(174,131))
--model:add(nn.Tanh())
model:add(nn.Linear(700,36))
model:add(nn.LogSoftMax())

In [173]:
criterion = nn.ClassNLLCriterion()

In [174]:
sgd_params = {
   learningRate = 1e-2,
   learningRateDecay = 1e-4,
   weightDecay = 1e-3,
   momentum = 1e-4
}

In [175]:
x, dl_dx = model:getParameters()

In [176]:
step = function(batch_size)
    local current_loss = 0
    local count = 0
    batch_size = batch_size or 200
    for t = 1, trainset.size, batch_size do
        local size = math.min(t + batch_size -1, trainset.size) - t
        local inputs = torch.Tensor(size, 15, 15)
        local targets = torch.Tensor(size)
        for i = 1,size do
            local input = trainset.data[i+t]
            local target = trainset.label[i+t]
            -- if target == 0 then target = 10 end
            inputs[i] = input
            targets[i] = target
        end
        
        local feval = function(x_new)
            -- reset data
            if x ~= x_new then x:copy(x_new) end
            dl_dx:zero()

            -- perform mini-batch gradient descent
            local loss = criterion:forward(model:forward(inputs), targets)
            model:backward(inputs, criterion:backward(model.output, targets))

            return loss, dl_dx
        end
        
        _, fs = optim.sgd(feval, x, sgd_params)
        -- fs is a table containing value of the loss function
        -- (just 1 value for the SGD optimization)
        count = count + 1
        current_loss = current_loss + fs[1]
    end

    -- normalize loss
    return current_loss / count
end

In [177]:
eval = function(dataset, batch_size)
    local count = 0
    batch_size = batch_size or 200
    
    for i = 1,dataset.size,batch_size do
        local size = math.min(i + batch_size - 1, dataset.size) - i
        local inputs = dataset.data[{{i,i+size-1}}]
        local targets = dataset.label[{{i,i+size-1}}]:long()
        local outputs = model:forward(inputs)
        local _, indices = torch.max(outputs, 2)
        local guessed_right = indices:eq(targets):sum()
        count = count + guessed_right
    end

    return count / dataset.size
end

In [306]:
max_iters = 100

In [ ]:
do
    local last_accuracy = 0
    local decreasing = 0
    local threshold = 1 -- how many deacreasing epochs we allow
    for i = 1,max_iters do
        local loss = step()
        print(string.format('Epoch: %d Current loss: %4f', i, loss))
        local accuracy = eval(validationset)
        print(string.format('Accuracy on the validation set: %4f', accuracy))
        if accuracy < last_accuracy then
            if decreasing > threshold then break end
            decreasing = decreasing + 1
        else
            decreasing = 0
        end
        last_accuracy = accuracy
    end
end

Epoch: 1 Current loss: 0.075341	


Accuracy on the validation set: 0.956564	


Epoch: 2 Current loss: 0.075301	


Accuracy on the validation set: 0.956564	


Epoch: 3 Current loss: 0.075263	


...

Accuracy on the validation set: 0.956564	


Epoch: 7 Current loss: 0.075128	


Accuracy on the validation set: 0.956564	


Epoch: 8 Current loss: 0.075097	


Accuracy on the validation set: 0.956564	


In [205]:
x, dl_dx = model:getParameters()
theta={x =x, dl_dx=dl_dx}
torch.save('theta3.dat',theta)

In [216]:
eval(testset)

0.96975	


In [217]:
eval(trainset)

0.9804375	


In [218]:
eval(validationset)

0.95656394079107	


In [296]:
a=image.load("r.png",1,'byte')
--a=testset.data[10]

In [297]:
b=torch.Tensor(1,15,15)
b[1]=image.scale(a,15,15)
itorch.image(b[1])

In [305]:
pred=model:forward(b)
conf, indi=torch.max(pred,2)
print (indi[1],torch.max(pred))

 28
[torch.LongTensor of size 1]

-0.0035636091200004	
